In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [2]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet_FC'
# NETWORK = 'Resnet'
# NETWORK = 'Resnet_res2c'
# NETWORK = 'Resnet_res3d'
# NETWORK = 'Resnet_res4f'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'
# NETWORK = 'EfficientNetV2M'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ResNet152V2'
NETWORK = 'ConvNeXtBase'

METHOD = "Max-Max"

In [3]:
WORKING_DIRECTORY = '/home/amirhosein/HECKTOR2022/WHOLEIMAGE_MAMIP'

PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE_Max-Median.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"ProcessedFeatures_{METHOD}")

processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE_{METHOD}.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/home/amirhosein/HECKTOR2022/hecktor2022_patient_endpoint_training.csv"

In [4]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,2.536475,3.008695,4.015192,4.186615,4.089388,2.377351,1.958764,3.389729,2.414497,...,2.974998,2.446435,2.479748,2.963974,2.479652,1.511370,2.236015,4.679839,3.679147,3.693898
1,CHUM-002,2.503484,2.690601,3.455625,4.765652,3.133198,1.978719,1.242311,3.379724,1.623484,...,3.056859,2.215354,3.072847,2.927061,2.727643,1.896733,2.668320,3.460523,4.143346,5.025620
2,CHUM-006,1.624664,3.011545,3.243611,4.715483,2.766528,1.891336,1.697049,2.634541,2.193834,...,2.512155,2.860037,2.914514,3.235997,2.351075,1.564157,1.968486,2.582835,5.403337,3.568925
3,CHUM-007,2.324913,2.584802,3.543478,5.384784,3.065820,1.802824,1.651898,2.889524,2.635265,...,2.771647,3.406075,3.009984,3.638395,1.791522,1.462965,2.101698,2.837297,3.772630,4.198152
4,CHUM-008,2.763511,2.743536,3.477528,5.474971,4.622638,2.513320,1.742555,2.956471,2.621997,...,2.534781,3.009434,1.809267,3.004713,1.982250,1.835523,1.924526,2.635923,3.785991,3.962818


# Up to here

Read Outcome csv

In [5]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [6]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [7]:
OD.columns[0]

'Patient_ID'

In [8]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [9]:
OD.columns[0]


'Patient_ID'

In [10]:
PET_outcome_data.columns[0]

'Patient_ID'

In [11]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,2.536475,3.008695,4.015192,4.186615,4.089388,2.377351,1.958764,3.389729,2.414497,...,2.974998,2.446435,2.479748,2.963974,2.479652,1.511370,2.236015,4.679839,3.679147,3.693898
1,CHUM-002,2.503484,2.690601,3.455625,4.765652,3.133198,1.978719,1.242311,3.379724,1.623484,...,3.056859,2.215354,3.072847,2.927061,2.727643,1.896733,2.668320,3.460523,4.143346,5.025620
2,CHUM-006,1.624664,3.011545,3.243611,4.715483,2.766528,1.891336,1.697049,2.634541,2.193834,...,2.512155,2.860037,2.914514,3.235997,2.351075,1.564157,1.968486,2.582835,5.403337,3.568925
3,CHUM-007,2.324913,2.584802,3.543478,5.384784,3.065820,1.802824,1.651898,2.889524,2.635265,...,2.771647,3.406075,3.009984,3.638395,1.791522,1.462965,2.101698,2.837297,3.772630,4.198152
4,CHUM-008,2.763511,2.743536,3.477528,5.474971,4.622638,2.513320,1.742555,2.956471,2.621997,...,2.534781,3.009434,1.809267,3.004713,1.982250,1.835523,1.924526,2.635923,3.785991,3.962818


In [12]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [13]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [14]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1016,1017,1018,1019,1020,1021,1022,1023,Relapse,RFS
0,CHUM-001,2.536475,3.008695,4.015192,4.186615,4.089388,2.377351,1.958764,3.389729,2.414497,...,2.479748,2.963974,2.479652,1.511370,2.236015,4.679839,3.679147,3.693898,0,1704
1,CHUM-002,2.503484,2.690601,3.455625,4.765652,3.133198,1.978719,1.242311,3.379724,1.623484,...,3.072847,2.927061,2.727643,1.896733,2.668320,3.460523,4.143346,5.025620,1,439
2,CHUM-006,1.624664,3.011545,3.243611,4.715483,2.766528,1.891336,1.697049,2.634541,2.193834,...,2.914514,3.235997,2.351075,1.564157,1.968486,2.582835,5.403337,3.568925,0,1186
3,CHUM-007,2.324913,2.584802,3.543478,5.384784,3.065820,1.802824,1.651898,2.889524,2.635265,...,3.009984,3.638395,1.791522,1.462965,2.101698,2.837297,3.772630,4.198152,0,1702
4,CHUM-008,2.763511,2.743536,3.477528,5.474971,4.622638,2.513320,1.742555,2.956471,2.621997,...,1.809267,3.004713,1.982250,1.835523,1.924526,2.635923,3.785991,3.962818,0,1499


In [15]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)